<a href="https://colab.research.google.com/github/Tiabet/Boat_Competition/blob/main/KBoat_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [3]:
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification

# Load the fine-tuned model and tokenizer
model = XLMRobertaForSequenceClassification.from_pretrained("drive/MyDrive/KBoatClassifier")
tokenizer = XLMRobertaTokenizer.from_pretrained("drive/MyDrive/KBoatClassifier_tok")

In [5]:
from datasets import load_dataset
import torch

# Define the paths to your JSONL files
train_file = "/content/drive/MyDrive/data/train_double_v2.jsonl"
val_file = "/content/drive/MyDrive/data/val_double_v2.jsonl"
test_file = "/content/drive/MyDrive/data/test_double_v2.jsonl"

# Load datasets
train_dataset = load_dataset('json', data_files=train_file, split='train')
val_dataset = load_dataset('json', data_files=val_file, split='train')
test_dataset = load_dataset('json', data_files=test_file, split='train')

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
# Define the paths to your JSONL files
train_file = "/content/drive/MyDrive/data/train_double_v2.jsonl"
val_file = "/content/drive/MyDrive/data/val_double_v2.jsonl"
test_file = "/content/drive/MyDrive/data/test_double_v2.jsonl"

# Load datasets
train_dataset = load_dataset('json', data_files=train_file, split='train')
val_dataset = load_dataset('json', data_files=val_file, split='train')
test_dataset = load_dataset('json', data_files=test_file, split='train')

In [7]:
def preprocess_function(examples):
    input_texts = []
    labels = []

    for message_set in examples['messages']:
        user_message = next(msg['content'] for msg in message_set if msg['role'] == 'user')
        assistant_message = next(msg['content'] for msg in message_set if msg['role'] == 'assistant')

        # Append the user's input as the text to be tokenized
        input_texts.append(user_message)

        # Label as 0 (for failure) or 1 (for success) based on the assistant's response
        if assistant_message == "실패":
            labels.append(0)
        else:
            labels.append(1)

    return {'text': input_texts, 'label': labels}

# Apply the preprocessing function to each dataset (train, val, test)
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/79950 [00:00<?, ? examples/s]

Map:   0%|          | 0/8952 [00:00<?, ? examples/s]

Map:   0%|          | 0/6102 [00:00<?, ? examples/s]

In [10]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# Tokenize the datasets
# tokenized_train = train_dataset.map(tokenize_function, batched=True)
# tokenized_val = val_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/6102 [00:00<?, ? examples/s]

In [11]:
from transformers import Trainer

trainer = Trainer(model=model)

predictions = trainer.predict(tokenized_test)

logits = predictions.predictions


In [12]:
probabilities = torch.nn.functional.softmax(torch.tensor(logits), dim=-1).numpy()

print(probabilities)

[[0.68749386 0.31250617]
 [0.687494   0.31250593]
 [0.687494   0.312506  ]
 ...
 [0.68749416 0.31250584]
 [0.6874941  0.31250584]
 [0.6874938  0.3125062 ]]


In [13]:
len(probabilities)

6102

In [14]:
grouped_probabilities = probabilities.reshape(-1, 6, probabilities.shape[1])

print("Grouped probabilities (by 6 rows):", grouped_probabilities)

Grouped probabilities (by 6 rows): [[[0.68749386 0.31250617]
  [0.687494   0.31250593]
  [0.687494   0.312506  ]
  [0.6874939  0.31250617]
  [0.6874939  0.31250608]
  [0.68749356 0.3125064 ]]

 [[0.68749416 0.31250584]
  [0.6874939  0.3125061 ]
  [0.68749356 0.3125064 ]
  [0.6874939  0.3125061 ]
  [0.68749434 0.3125057 ]
  [0.68749386 0.3125062 ]]

 [[0.6874937  0.31250635]
  [0.6874939  0.31250614]
  [0.687494   0.312506  ]
  [0.6874941  0.3125059 ]
  [0.68749386 0.31250617]
  [0.6874937  0.31250632]]

 ...

 [[0.687494   0.31250593]
  [0.687494   0.31250596]
  [0.6874938  0.3125062 ]
  [0.687494   0.312506  ]
  [0.687494   0.31250608]
  [0.68749386 0.31250617]]

 [[0.6874938  0.3125062 ]
  [0.6874939  0.3125061 ]
  [0.6874939  0.3125061 ]
  [0.687494   0.31250608]
  [0.68749374 0.3125063 ]
  [0.6874938  0.31250623]]

 [[0.68749416 0.31250584]
  [0.6874937  0.31250632]
  [0.68749374 0.31250626]
  [0.68749416 0.31250584]
  [0.6874941  0.31250584]
  [0.6874938  0.3125062 ]]]


In [15]:
import pandas as pd
import numpy as np

column_names = [f'선수{i+1}' for i in range(6)]
df = pd.DataFrame(grouped_probabilities[:, :, 1], columns=column_names)

In [17]:
df

,선수1,선수2,선수3,선수4,선수5,선수6
0,0.312506,0.312506,0.312506,0.312506,0.312506,0.312506
1,0.312506,0.312506,0.312506,0.312506,0.312506,0.312506
2,0.312506,0.312506,0.312506,0.312506,0.312506,0.312506
3,0.312506,0.312506,0.312506,0.312506,0.312506,0.312506
4,0.312506,0.312506,0.312506,0.312506,0.312506,0.312506
...,...,...,...,...,...,...
1012,0.312506,0.312506,0.312506,0.312506,0.312506,0.312506
1013,0.312506,0.312506,0.312507,0.312506,0.312506,0.312506
1014,0.312506,0.312506,0.312506,0.312506,0.312506,0.312506
1015,0.312506,0.312506,0.312506,0.312506,0.312506,0.312506
